In [1]:
# import necessary library
import numpy as np
from phasepy import component, mixture, preos
from phasepy.equilibrium import flash

In [2]:
# given data
F = 1000 # molar flow. kmol/h
T = 87. + 273.15  # vessel temperature, converted to K
P = 1.01 # vessel pressure, bar
Z = np.array([0.75, 0.25]) # overall molar fraction component

In [3]:
# define component thermodynamical properties
benzene = component(name='benzene', Tc=562.2, Pc=48.98, Zc=0.271, Vc=259.0,
                        w=0.210,
                        Ant=[13.7819, 2726.81, 217.572],
                        GC={'CH=C':6})

toluene = component(name='toluene', Tc=591.8, Pc=41.06, Zc=0.264, Vc=316,
                      w=0.262,
                      Ant=[13.9320, 3056.96, -217.625],
                      GC={'CH=C':6, 'CH3':1})

In [4]:
# setting up eos
mix = mixture(benzene, toluene) #mix given component
mix.unifac() # using dortmund modified unifac mixing rule
eos = preos(mix, 'mhv_unifac') # peng robinson equation of state

In [5]:
# initial guess
x0 = np.array([0.4, 0.6]) # liquid molar fraction to guess
y0 = np.array([0.2, 0.8]) # vapor molar fraction to guess

In [6]:
# start calculation
sep = flash(x0, y0, 'VL', Z, T, P, eos) # solver for flash calculation

In [7]:
sep

(array([0.81063167, 0.18936833]),
 array([0.64528823, 0.35471177]),
 0.3667014050359152)

In [8]:
y1 = (sep[0])[0] # benzene vapor fraction, indexing from tuple
y2 = (sep[0])[1] # toluene vapor fraction, indexing from tuple
x1 = (sep[1])[0] # benzene liquid fraction, indexing from tuple
x2 = (sep[1])[1] # benzene liquid fraction, indexing from tuple

In [9]:
y1, y2, x1, x2

(0.8106316699864328,
 0.18936833001356734,
 0.6452882348270927,
 0.35471176517290726)

In [10]:
# compute vapor flow and liquid flow using matrix
import scipy.linalg # import new library
A = np.array ([[y1, x1],[y2, x2]])
B = np.array ([F*Z[0], F*Z[1]])

In [11]:
print (A)
print (B)

[[0.81063167 0.64528823]
 [0.18936833 0.35471177]]
[750. 250.]


In [12]:
# now solve the matrix
C = scipy.linalg.solve(A,B) # solver for linear algebra

In [13]:
print (C)

[633.29859496 366.70140504]


In [14]:
# print all solution
print ("----------------------------------------------------")
print ("Vapor Flow (V) :", C[0], "kmol/h")
print ("     benzene fraction in vapor (y1) :", y1)
print ("     toluene fraction in vapor (y2) :", y2)
print ("----------------------------------------------------")
print ("Liquid Flow (L) :", C[1], "kmol/h")
print ("     benzene fraction in liquid (x1) :", x1)
print ("     toluene fraction in liquid (x2) :", x2)

----------------------------------------------------
Vapor Flow (V) : 633.2985949638551 kmol/h
     benzene fraction in vapor (y1) : 0.8106316699864328
     toluene fraction in vapor (y2) : 0.18936833001356734
----------------------------------------------------
Liquid Flow (L) : 366.70140503614493 kmol/h
     benzene fraction in liquid (x1) : 0.6452882348270927
     toluene fraction in liquid (x2) : 0.35471176517290726
